In [11]:
import numpy as np
import seaborn as sns
from copy import deepcopy
import os
from pathlib import Path
from tqdm.auto import tqdm

In [12]:
root = Path("/home/edu/code/google_fast_or_slow/data/npz_all/npz")
collection = "layout/xla"
ctype = "default"

In [13]:
def prune_graph(data):
    print("Pruning graph...")
    new_data = deepcopy(dict(data))
    print("Original graph has {} nodes and {} edges".format(data["node_feat"].shape[0], data["edge_index"].shape[0]))
    in_edge_index = data["edge_index"][np.isin(data["edge_index"], data["node_config_ids"]).any(1)]

    in_node_ids = np.unique(in_edge_index)
    assert len(set(data["node_config_ids"]) - set(in_node_ids)) == 0
    lookup = np.ones(data["node_feat"].shape[0]) * -1
    lookup[in_node_ids] = np.arange(in_node_ids.shape[0])

    in_node_feats = data["node_feat"][in_node_ids, :]
    in_node_opcode = data["node_opcode"][in_node_ids]
    in_edge_index = lookup[in_edge_index]
    in_node_config_ids = lookup[data["node_config_ids"]]

    new_data["node_feat"] = in_node_feats
    new_data["node_opcode"] = in_node_opcode
    new_data["edge_index"] = in_edge_index
    new_data["node_config_ids"] = in_node_config_ids
    print("New graph has {} nodes and {} edges".format(new_data["node_feat"].shape[0], new_data["edge_index"].shape[0]))
    return new_data


In [14]:
def remove_dupplicated_node_configs(data):
    reshaped_config_feat = data["node_config_feat"].reshape(data["node_config_feat"].shape[0], -1) + 2 # avoid zeros
    positional_array = np.random.random(reshaped_config_feat.shape[1])  # multiply each value by its position to avoid removing permutations by accident
    reshaped_values = (reshaped_config_feat * positional_array[None, :]).sum(1)
    is_equal_matrix = reshaped_values[None, :] == reshaped_values[:, None] # quadratic matrix of all pairwise equalities
    # is_equal_matrix[np.triu_indices(is_equal_matrix.shape[0], 0)] = 0 # only get diagonal to avoid remove twice
    is_equal_matrix = np.tril(is_equal_matrix, -1) # only get diagonal to avoid remove twice
    to_remove_ids = np.unique(np.where(is_equal_matrix)[0])
    print("Removing {} duplicated node configs out of {}".format(to_remove_ids.shape[0], data["node_config_feat"].shape[0]))
    data["config_runtime"] = np.delete(data["config_runtime"], to_remove_ids)
    data["node_config_feat"] = np.delete(data["node_config_feat"], to_remove_ids, axis=0)
    return data

In [15]:
dst_dir = root / f"{collection}_pruned" / ctype
for split in ["train", "valid", "test"]:
    print("Loading {} data...".format(split))
    split_src_dir = root / collection / ctype / split
    split_dst_dir = dst_dir / split
    split_dst_dir.mkdir(parents=True, exist_ok=True)

    for npz_path in tqdm(list(split_src_dir.glob("*.npz"))):
        print(npz_path)
        data = dict(np.load(str(npz_path), allow_pickle=True))
        data = prune_graph(data)
        if split == "train":
            data = remove_dupplicated_node_configs(data)
        np.savez(split_dst_dir / npz_path.name, **data)

Loading train data...


  0%|          | 0/61 [00:00<?, ?it/s]

/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/train/mlperf_transformer.npz
Pruning graph...
Original graph has 13502 nodes and 22209 edges
New graph has 2143 nodes and 2137 edges
Removing 384 duplicated node configs out of 5192


  2%|▏         | 1/61 [00:00<00:39,  1.52it/s]

/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/train/ncf.2x2.fp32.npz
Pruning graph...
Original graph has 490 nodes and 729 edges
New graph has 65 nodes and 56 edges


...

/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/test/db59a991b7c607634f13570d52ce885f.npz
Pruning graph...
Original graph has 5810 nodes and 10345 edges
New graph has 594 nodes and 599 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/test/5335ed13823b0a518ee3c79ba4425f34.npz
Pruning graph...
Original graph has 43615 nodes and 73881 edges
New graph has 3142 nodes and 3089 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/test/cd708819d3f5103afd6460b15e74eaf3.npz
Pruning graph...
Original graph has 490 nodes and 749 edges
New graph has 118 nodes and 116 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/test/fbaa8bb6a1aed9988281085c91065c05.npz


 75%|███████▌  | 6/8 [00:00<00:00, 11.81it/s]

Pruning graph...
Original graph has 24790 nodes and 32709 edges
New graph has 6411 nodes and 4774 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/test/3e7156ac468dfb75cf5c9615e1e5887d.npz
Pruning graph...


100%|██████████| 8/8 [00:00<00:00,  9.62it/s]

Original graph has 41522 nodes and 72902 edges
New graph has 6959 nodes and 6466 edges
/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/test/05ae41e26dd3c4c06390371a0423233c.npz
Pruning graph...
Original graph has 43615 nodes and 73881 edges
New graph has 3142 nodes and 3089 edges


In [ ]:
split_src_dir

In [3]:
path = "/home/edu/code/google_fast_or_slow/data/npz_all/npz/layout/xla/default/train/ncf.2x2.fp32.npz"
data = np.load(path)

In [4]:
list(data.keys())

['edge_index',
 'node_feat',
 'node_opcode',
 'node_config_feat',
 'node_config_ids',
 'node_splits',
 'config_runtime']

In [5]:
data["node_config_feat"].shape

(100040, 20, 18)

In [6]:
in_edge_index = data["edge_index"][np.isin(data["edge_index"], data["node_config_ids"]).any(1)]

in_node_ids = np.unique(in_edge_index)

In [10]:
len(set(data["node_config_ids"]) - set(in_node_ids))

0

In [ ]:
100040 - 99668

In [ ]:
reshaped_config_feat = data["node_config_feat"].reshape(data["node_config_feat"].shape[0], -1)
print(1)
positional_array = (np.arange(reshaped_config_feat.shape[1]) + 1) # multiply each value by its position to avoid removing permutations by accident
reshaped_values = (reshaped_config_feat * positional_array[None, :]).sum(1)
print(2)
is_equal_matrix = reshaped_values[None, :] == reshaped_values[:, None] # quadratic matrix of all pairwise equalities
print(3)
is_equal_matrix = np.tril(is_equal_matrix, -1) # only get diagonal to avoid remove twice
print(4)
to_remove_ids = np.unique(np.where(is_equal_matrix)[0])
print(5)
print("Removing {} duplicated node configs".format(to_remove_ids.shape[0]))

In [ ]:
remove_dupplicated_node_configs(data)

In [ ]:
(100040 ** 2) * 4 / 1e9

In [ ]:
remove_dupplicated_node_configs(dict(data))

In [ ]:
data["node_config_feat"].shape

In [ ]:
np.delete(x, to_remove_ids)


In [ ]:
is_equal_matrix.sum(1)

In [ ]:
x = np.array([1, 1, 2, 3, 3, 4, 7, 1, 2])
m = x[None, :] == x[:, None]
# remove 1,4, 7, 8

In [ ]:
m

In [ ]:
np.tril(m, -1)

In [ ]:

m

In [ ]:
to_remove_ids = np.unique(np.where(m)[0])

In [ ]:
np.delete(x, to_remove_ids)

In [ ]:
(5304 ** 2) / 1e9

In [ ]:
new_data = prune_graph(data)

In [ ]:
data["edge_index"].shape#[3, :]

In [ ]:

in_node_feats.shape

In [ ]:
data["node_feat"].shape

In [ ]:
np.unique(data["node_config_feat"][:, :, 98], return_counts=True)

In [ ]:
{k: data[k].shape for k in data.keys()}

In [ ]:
np.unique((data["node_config_feat"].sum(2) != -18).sum(1), return_counts=True)

In [ ]:
sns.histplot((data["node_config_feat"].sum(2) != -18).sum(1))

In [ ]:
((data["node_config_feat"][0] != data["node_config_feat"][3]).sum(1) > 0).sum()

In [ ]:
100040 * 100040 * 1 / 1e9

In [ ]:
from tqdm.auto import tqdm

In [ ]:
data["node_config_feat"][i:i+1].shape

In [ ]:
((data["node_config_feat"][i:i+1] != data["node_config_feat"]).sum(2) > 0).sum(1)

In [ ]:
max_size = 100
bz = 100
result = np.zeros((max_size, 100040), dtype=np.uint8)
for i in tqdm(range(0, max_size, bz)):
    result[i:i+bz] = ((data["node_config_feat"][i:i+bz, None] != data["node_config_feat"][None, ...]).sum(-1) > 0).sum(-1)

In [ ]:
result.max(1)

In [ ]:
(result == 1).sum(1)

In [ ]:
(result == 0).sum(1)

In [ ]:
result.min(1)

In [ ]:
np.median(result, 1)

In [ ]:
np.percentile(result, 0.1, 1)

In [ ]:
(data["node_config_feat"][0] != data["node_config_feat"][0][1]).sum()

In [ ]:
np.unique(data["node_feat"][:, 22])